# 風險地圖

In [1]:
import os
import pandas as pd
import folium
import json
import geopandas
from folium import plugins
from folium.plugins import MarkerCluster, Search
from folium.plugins import HeatMap
from json2html import *
from geojson_utils import *
from bs4 import BeautifulSoup
import numpy as np
import html
data = pd.read_csv("106年火災事件資料.csv")
GeoJsonArr = [file for file in os.listdir("./建築執照資料整理") if file.endswith("geo.json")]
len(GeoJsonArr)

21

In [2]:
GeoJsonArr[1][:3]

'臺中市'

## 統計各區域火災數量 (106年)

In [3]:
data_modified = data[(data["縣市"]!="高雄港") & (data["縣市"]!="臺中港")& (data["縣市"]!="基隆港")]
df = pd.DataFrame()
df["區域"] = pd.Series(data_modified["鄉_鎮_市_區"].value_counts().index.values)
df["數量"] = pd.Series(data_modified["鄉_鎮_市_區"].value_counts().values)
df

,區域,數量
0,沙鹿區,401
1,南區,393
2,安南區,381
3,彰化市,350
4,中山區,347
...,...,...
350,鹿野鄉,2
351,茂林區,2
352,鹿谷鄉,1
353,泰武鄉,1


In [4]:
for 縣市 in GeoJsonArr:
  data_modified = data[data["縣市"]==縣市[:3]]
  df = pd.DataFrame()
  df["區域"] = pd.Series(data_modified["鄉_鎮_市_區"].value_counts().index.values)
  df["數量"] = pd.Series(data_modified["鄉_鎮_市_區"].value_counts().values)
  data_car_A1 = pd.read_csv(縣市[:3]+"_A1.csv")
  data_car_A2 = pd.read_csv(縣市[:3]+"_A2.csv")
  data_car_pos_A1 = data_car_A1[["經度","緯度","車種","死亡受傷人數"]]
  data_car_pos_A1 = data_car_A1.dropna(axis=0, how='any')
  data_car_pos_A2 = data_car_A2[["經度","緯度","車種","死亡受傷人數"]]
  data_car_pos_A2 = data_car_A2.dropna(axis=0, how='any')
  Total_map = folium.Map(location=[23.58,121], zoom_start=7)
  marker_cluster = MarkerCluster(name="A1車禍統計").add_to(Total_map)
  with open(縣市[:3]+"_區域界線.json", "r") as f:
      multipoly = json.load(f)

  folium.Choropleth(
      geo_data=縣市[:3]+"_區域界線.json",
      name='火災事件',
      data = df,
      fill_color='YlGnBu',
      fill_opacity=0.5,
      columns=["區域","數量"],
      key_on='feature.properties.T_Name',
  ).add_to(Total_map)

  死亡人數 = []
  受傷人數 = []
  for item in data_car_pos_A1["死亡受傷人數"].str[:].values:
    死亡人數.append(float(item.split(";")[0][2:]))
    受傷人數.append(float(item.split(";")[1][2:]))
  for item in data_car_pos_A2["死亡受傷人數"].str[:].values:
    死亡人數.append(float(item.split(";")[0][2:]))
    受傷人數.append(float(item.split(";")[1][2:]))
  嚴重程度 =  np.array(死亡人數)*10 + np.array(受傷人數)
  車禍 = zip(np.concatenate((data_car_pos_A1["緯度"].values,data_car_pos_A2["緯度"].values)),
          np.concatenate((data_car_pos_A1["經度"].values,data_car_pos_A2["經度"].values)),
          嚴重程度)
  HeatMap(車禍,radius = 12).add_to(folium.FeatureGroup(name="Heat Map").add_to(Total_map))

  with open("./建築執照資料整理/"+縣市, "r") as f:
      JSON_file = json.load(f)
  building= geopandas.read_file("./建築執照資料整理/"+縣市)
  addressgeo = folium.GeoJson(
    JSON_file,
    name='建築位置',
    show=False
  ).add_to(Total_map)

  statesearch = Search(
    layer=addressgeo,
    geom_type='Points',
    placeholder='搜尋地址',
    collapsed=False,
    search_label='地址',
    search_zoom=17,
    weight=3,
    #tooltip=folium.GeoJsonTooltip(fields=["地址"],aliases=["地址"])
  ).add_to(Total_map)

  marker_cluster = MarkerCluster(name="建築標記").add_to(Total_map)
  address_list=[]

  for i in range(len(building)):
      try:
          popuptext = "<h4>{}</h4>{}".format(JSON_file["features"][i]["properties"]["地址"]
                                           ,json2html.convert(json = JSON_file["features"][i]["properties"]["樓層概要"]))
      except:
          popuptext = "<h4>{}</h4>".format(JSON_file["features"][i]["properties"]["地址"])
      test = folium.Html(popuptext, script=True)
      popup = folium.Popup(test, max_width=300,min_width=300)
      folium.Marker(
          popup = popup,
          location=[building["經緯"].values[i].split(",")[0],building["經緯"].values[i].split(",")[1]],
          icon=folium.Icon(color='darkgreen', icon='home', prefix='fa')).add_to(marker_cluster)

  folium.LayerControl().add_to(Total_map)


  Total_map.save(縣市[:3]+'.html')

In [5]:
GeoJsonArr[0:1]

['桃園市107_geo.json']

In [6]:
for 縣市 in GeoJsonArr:
  html_file = open(縣市[:3]+".html").read()
  soup = BeautifulSoup(html_file)
  script = """
  <!-- Core Build -->
  <script src="https://cdn-geoweb.s3.amazonaws.com/esri-leaflet/1.0.0-rc.2/esri-leaflet-core.js"></script>

  <!-- Basemaps Only Build -->
  <script src="https://cdn-geoweb.s3.amazonaws.com/esri-leaflet/1.0.0-rc.2/esri-leaflet-basemaps.js"></script>

  <!-- Feature Layer Only Build -->
  <script src="https://cdn-geoweb.s3.amazonaws.com/esri-leaflet/1.0.0-rc.2/esri-leaflet-feature-layer.js"></script>

  <!-- Map Service Only Build -->
  <script src="https://cdn-geoweb.s3.amazonaws.com/esri-leaflet/1.0.0-rc.2/esri-leaflet-map-service.js"></script>

  <!-- Heatmap Feature Layer -->
  <script src="https://cdn-geoweb.s3.amazonaws.com/esri-leaflet-heatmap-feature-layer/1.0.0-rc.2/esri-leaflet-heatmap-feature-layer.js"></script>

  <!-- Clustered Feature Layer -->
  <script src="https://cdn-geoweb.s3.amazonaws.com/esri-leaflet-clustered-feature-layer/1.0.0-rc.2/esri-leaflet-clustered-feature-layer.js"></script>

  <!-- Geocoding Control -->
  <script src="https://cdn-geoweb.s3.amazonaws.com/esri-leaflet-geocoder/0.0.1-beta.5/esri-leaflet-geocoder.js"></script>
  <link rel="stylesheet" type="text/css" href="http://cdn-geoweb.s3.amazonaws.com/esri-leaflet-geocoder/0.0.1-beta.5/esri-leaflet-geocoder.css">
  """
  script2 = """var searchControl = new L.esri.Controls.Geosearch().addTo(map);

    var results = new L.LayerGroup().addTo(map);

    searchControl.on('results', function(data){
      /*results.clearLayers();*/
      for (var i = data.results.length - 1; i >= 0; i--) {
        results.addLayer(L.marker(data.results[i].latlng));
      }
    });
  """
  soup.head.style.insert_after(soup.new_string(script.replace("map",soup.body.div["id"])))
  with open(縣市[:3]+"_search.html", "w") as f:
    f.write(html.unescape(str(soup)[:-16])+
            script2.replace("map",soup.body.div["id"])+
            html.unescape(str(soup)[-16:]))